In [1]:
import torch
import numpy as np

In [2]:
def onehot_encoder(position,depth):
    onehot=torch.zeros((depth,))
    onehot[position]=1
    return onehot

def int_to_onehot(number):
  onehot=onehot_encoder(number, 100)
  return onehot

In [3]:
# Function to generate 10 ints (between 0 & 99) that are multiple of 5
def gen_sequence():
    indices = torch.randint(0, 20, (10,))
    sequence = indices*5
    return sequence

sequence = gen_sequence()
print(sequence)

tensor([60, 65, 40, 35, 95, 80, 40, 50, 45, 65])


In [4]:
# Convert each int in preceding step into one-hot var
def gen_batch():
  sequence = gen_sequence()
  batch=[int_to_onehot(i).numpy() for i in sequence]
  batch=np.array(batch)
  return torch.tensor(batch)

batch=gen_batch()
print(batch)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         

In [5]:
def data_to_num(data):
    num=torch.argmax(data,dim=-1)
    return num
numbers=data_to_num(batch)
print(numbers)

tensor([65, 90, 25, 55, 55, 40, 50, 45, 60, 95])


In [6]:
from torch import nn

device="cuda" if torch.cuda.is_available() else "cpu"

# Discriminator
D=nn.Sequential(
    nn.Linear(100,1),
    nn.Sigmoid()).to(device)

In [7]:
# Generator

G=nn.Sequential(
    nn.Linear(100,100),
    nn.ReLU()).to(device)


In [8]:
loss_fn=nn.BCELoss()
lr=0.0005
optimD=torch.optim.Adam(D.parameters(),lr=lr)
optimG=torch.optim.Adam(G.parameters(),lr=lr)

In [9]:
real_labels=torch.ones((10,1)).to(device)
fake_labels=torch.zeros((10,1)).to(device)

In [10]:
def train_D_G(D,G,loss_fn,optimD,optimG):
    # Generate examples of real data
    true_data=gen_batch().to(device)

    # use 1 as labels since they are real
    preds=D(true_data)
    loss_D1=loss_fn(preds,real_labels.reshape(10,1))
    optimD.zero_grad()
    loss_D1.backward()
    optimD.step()

    # train D on fake data
    noise=torch.randn(10,100).to(device)
    generated_data=G(noise)

    # use 0 as labels since they are fake
    preds=D(generated_data)
    loss_D2=loss_fn(preds,fake_labels.reshape(10,1))
    optimD.zero_grad()
    loss_D2.backward()
    optimD.step()

    # train G
    noise=torch.randn(10,100).to(device)
    generated_data=G(noise)

    # use 1 as labels since G wants to fool D
    preds=D(generated_data)
    loss_G=loss_fn(preds,real_labels.reshape(10,1))
    optimG.zero_grad()
    loss_G.backward()
    optimG.step()
    return generated_data

In [11]:
class EarlyStop:
    def __init__(self, patience=1000):    #A
        self.patience = patience
        self.steps = 0
        self.min_gdif = float('inf')
    def stop(self, gdif):    #B
        if gdif < self.min_gdif:    #C
            self.min_gdif = gdif
            self.steps = 0
        elif gdif >= self.min_gdif:
            self.steps += 1
        if self.steps >= self.patience:    #D
            return True
        else:
            return False

stopper=EarlyStop(800)    #A

mse=nn.MSELoss()
real_labels=torch.ones((10,1)).to(device)
fake_labels=torch.zeros((10,1)).to(device)
def distance(generated_data):    #B
    nums=data_to_num(generated_data)
    remainders=nums%5
    ten_zeros=torch.zeros((10,1)).to(device)
    mseloss=mse(remainders,ten_zeros)
    return mseloss

for i in range(10000):
    gloss=0
    dloss=0
    generated_data=train_D_G(D,G,loss_fn,optimD,optimG)    #C
    dis=distance(generated_data)
    if stopper.stop(dis)==True:
        break
    if i % 50 == 0:
        print(data_to_num(generated_data))    #D

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([ 2, 42, 20, 66, 58,  9, 13, 21, 38, 39])
tensor([34, 20, 63, 33,  1, 21,  9, 44, 77, 66])
tensor([67, 21,  8, 20, 36, 90, 75, 93, 52, 93])
tensor([53, 17, 70, 17, 69, 46, 21, 69, 54, 44])
tensor([29, 44, 70, 12, 42, 70, 17, 33, 19, 50])
tensor([ 0,  4, 85, 12, 85, 20, 85, 85, 66, 68])
tensor([53, 53, 11, 73, 66, 22, 16, 67, 44, 70])
tensor([68, 20, 66, 68, 68,  1, 69, 81, 85,  6])
tensor([85, 85, 85, 70, 85, 39, 85, 15, 85, 22])
tensor([92, 80, 70, 66, 69, 20, 61, 70, 20,  4])
tensor([66, 66, 55,  0, 20, 85, 85, 62, 66, 70])
tensor([70,  0, 85, 13, 70, 67,  4, 20, 85, 70])
tensor([66, 75, 10, 30, 85, 32, 70, 70, 75, 80])
tensor([75, 11, 40, 20, 80, 45,  0, 85, 80, 20])
tensor([15, 25, 75, 25, 55, 10, 25, 60, 15, 80])
tensor([70, 25, 25, 80, 80, 75, 80, 80, 75, 30])
tensor([45, 25, 75, 25, 80, 40, 30, 25, 25, 15])
tensor([45, 25, 40, 55, 40, 50, 25, 95, 40, 35])
tensor([40, 25, 40, 45, 35, 30, 15, 75, 55, 45])
tensor([10, 55,  0, 45, 30, 25, 10, 25, 15, 30])
tensor([95, 10, 95, 